In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.nn.modules.loss import BCEWithLogitsLoss, BCELoss
import rasterio
import numpy as np
import matplotlib
import os
import glob
import matplotlib.pyplot as plt
import cv2
from IPython.core.pylabtools import figsize
from Model.load_data import CustomGeoDataset
from Model.unet import unet
import segmentation_models_pytorch as smp

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#DEVICE = 'cpu'
print("[INFO] training using {} ...".format(DEVICE))

[INFO] training using cuda ...


In [3]:
Image_Drive = "./Data/train/images"
Label_Drive = "./Data/train/labels"

#augmented data
#Image_aug = "./Data/Transformed_images"
#Label_aug = "./Data/Transformed_labels"

image_path = glob.glob(os.path.join(Image_Drive, "*.tif"))
label_path = glob.glob(os.path.join(Label_Drive, "*.tif"))


image_size = len(image_path)
label_size = len(label_path)

print(image_size)
print(label_size)

2457
2457


aug_image_path = glob.glob(os.path.join(Image_aug, "*.tif"))
aug_label_path = glob.glob(os.path.join(Label_aug, "*.tif"))

image_path = image_path + aug_image_path
label_path = label_path + aug_label_path

image_size = len(image_path)
label_size = len(label_path)

print(image_size)
print(label_size)

In [4]:
# Instantiate the custom dataset with transformations
train_data = CustomGeoDataset(image_path, label_path)
print(len(train_data))

split_size = 0.80
train_size = int(split_size * len(train_data))
test_size = len(train_data) - train_size
train_data, test_data = torch.utils.data.random_split(train_data, [train_size, test_size])

#dataloader
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=8)
testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=8)

print(len(trainloader))
print(len(testloader))

2457
246
62


In [5]:
def make_train_step(model, optimizer, loss_fn):
    def train_step(x, y):
        # Ensure x has the correct shape
        if x.dim() == 3:
            x = x.unsqueeze(1)  # Add channel dimension
        
        # Make prediction
        yhat = model(x)
        # Enter train mode
        model.train()
        # Compute loss
        loss = loss_fn(yhat, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        return loss
    return train_step

In [ ]:
# model unetransformer
#modelunetplusplus = smp.UnetPlusPlus(in_channels=4, encoder_depth=5, classes=1,
#    activation=None,
#    encoder_weights="imagenet",
#    encoder_name="efficientnet-b4",
#    decoder_attention_type="scse")

modelunetplusplus = smp.UnetPlusPlus(in_channels=4, encoder_depth=5, classes=1, encoder_name="efficientnet-b4", decoder_attention_type="scse")


# Define the U-Net model
model_unet = smp.Unet(
    encoder_name="efficientnet-b4",     # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                    # Choose depth for encoder
    in_channels=4,                      # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1,                          # Model output channels (number of classes)
    decoder_attention_type="scse"       # Optional, add SCSE attention module to decoder
)

# Define the DeepLabV3+ model
model_deeplabv3plus = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",     # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                    # Choose depth for encoder
    in_channels=4,                      # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                           # Model output channels (number of classes)
)

# Define the FPN model
model_fpn = smp.FPN(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                 # Choose depth for encoder
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                        # Model output channels (number of classes)
)

# Define the PSPNet model
model_pspnet = smp.PSPNet(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                 # Choose depth for encoder
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                        # Model output channels (number of classes)
)

# Define the PAN model
model_pan = smp.PAN(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                       # Model output channels (number of classes)
)

# Define the PAN model
model_manet = smp.MAnet(
    encoder_name="efficientnet-b4",  # Choose the encoder, e.g., efficientnet-b4
    encoder_depth=5,                 # Choose depth for encoder
    in_channels=4,                   # Model input channels (1 for grayscale, 3 for RGB, etc.)
    classes=1                        # Model output channels (number of classes)
)


modelunetplusplus = modelunetplusplus.to(DEVICE)
model_unet = model_unet.to(DEVICE)
model_pan = model_pan.to(DEVICE)
model_pspnet = model_pspnet.to(DEVICE)
model_fpn = model_fpn.to(DEVICE)
model_deeplabv3plus = model_deeplabv3plus.to(DEVICE)
model_manet = model_manet.to(DEVICE)

UnetPlusPlus(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      4, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticS

In [ ]:
from torch.optim import SGD

# define training hyperparameters
INIT_LR = 1e-4
epochs = 250

# initialize our optimizer and loss function
optimizer = SGD(modelunetplusplus.parameters(), lr=INIT_LR)
lossfn = BCEWithLogitsLoss()

#train step
train_step = make_train_step(modelunetplusplus, optimizer, lossfn)
print(train_step)

<function make_train_step.<locals>.train_step at 0x000002AB3AABB240>


In [11]:
from tqdm import tqdm
from torchmetrics import JaccardIndex

def train(model, trainloader, optimizer, loss_fn, device):
    model.train()
    train_step = make_train_step(model, optimizer, loss_fn)
    epoch_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    jaccard = JaccardIndex(task='binary').to(DEVICE)

    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        x_batch, y_batch = data
        x_batch = x_batch.float().to(device)  # Ensure correct type
        y_batch = y_batch.float().to(device)

        # Forward pass and loss calculation
        loss = train_step(x_batch, y_batch)
        epoch_loss += loss.item()

        # Predictions
        predictions = (model(x_batch) > 0.5).float()  # Binary thresholding at 0.5

        # Accuracy calculation
        correct_predictions += torch.sum(predictions == y_batch).item()
        total_predictions += (y_batch.size(0) * y_batch.size(1) * y_batch.size(2) * y_batch.size(3))

        # jaccard index (IoU) calculation
        jaccard.update(predictions, y_batch)

    accuracy = (correct_predictions/total_predictions) * 100  # Convert to percentage
    average_loss = epoch_loss / len(trainloader)
    iou = jaccard.compute()

    return average_loss, accuracy, iou

def evaluate(model, testloader, loss_fn, device):
    model.eval()
    epoch_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    jaccard = JaccardIndex(task='binary').to(DEVICE)

    with torch.no_grad():
        for x_batch, y_batch in testloader:
            x_batch = x_batch.float().to(device)
            y_batch = y_batch.float().to(device)

            # Forward pass and loss calculation
            yhat = model(x_batch)
            loss = loss_fn(yhat, y_batch)
            epoch_loss += loss.item()

            # Predictions
            predicted_classes = (yhat > 0.5).float() # Round to get binary predictions
            correct_predictions += torch.sum(predicted_classes == y_batch).item()
            total_predictions += (y_batch.size(0) * y_batch.size(1) * y_batch.size(2) * y_batch.size(3))

            #jaccard calculation
            jaccard.update(predicted_classes, y_batch)

    accuracy = (correct_predictions / total_predictions) * 100  # Convert to percentage
    average_loss = epoch_loss / len(testloader)
    iou = jaccard.compute()

    return average_loss, accuracy, iou


In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        Args:
            patience (int): Berapa banyak epoch untuk menunggu sebelum berhenti ketika tidak ada peningkatan.
            delta (float): Minimum perubahan pada monitored metric untuk dianggap peningkatan.
        """
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.best_model_wts = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_wts = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
best_loss3 = float('inf')
best_model_wts3 = None

train_losses3 = []
train_accuracies3 = []
test_losses3 = []
test_accuracies3 = []
train_ious3 = []
test_ious3 = []

# Define patience
PATIENCE = 10
# Initialize early stopping
early_stopping = EarlyStopping(patience=PATIENCE)

for epoch in range(epochs):
    train_loss, train_accuracy, train_iou = train(modelunetplusplus, trainloader, optimizer, lossfn, DEVICE)
    test_loss, test_accuracy, test_iou = evaluate(modelunetplusplus, testloader, lossfn, DEVICE)

    train_losses3.append(train_loss)
    train_accuracies3.append(train_accuracy)
    test_losses3.append(test_loss)
    test_accuracies3.append(test_accuracy)
    train_ious3.append(train_iou)
    test_ious3.append(test_iou)

    print(f'Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train IOU: {train_iou:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test IOU: {test_iou:.4f}')

    # Early stopping check
    early_stopping(test_loss, modelunetplusplus)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

    # Save the best model
    if test_loss < best_loss3:
        best_loss3 = test_loss

        best_model_wts3 = modelunetplusplus.state_dict()

modelunetplusplus.load_state_dict(best_model_wts3)


100%|██████████| 246/246 [01:05<00:00,  3.74it/s]


Epoch 1/250: Train Loss: 0.5270, Train Accuracy: 70.8177, Train IOU: 0.4948, Test Loss: 0.5036, Test Accuracy: 72.4749, Test IOU: 0.5090


100%|██████████| 246/246 [01:04<00:00,  3.83it/s]


Epoch 2/250: Train Loss: 0.5255, Train Accuracy: 70.7849, Train IOU: 0.4920, Test Loss: 0.5054, Test Accuracy: 72.3521, Test IOU: 0.5064


100%|██████████| 246/246 [01:04<00:00,  3.80it/s]


...

100%|██████████| 246/246 [01:27<00:00,  2.81it/s]


Epoch 249/250: Train Loss: 0.3761, Train Accuracy: 82.6075, Train IOU: 0.6786, Test Loss: 0.3330, Test Accuracy: 85.4442, Test IOU: 0.7170


100%|██████████| 246/246 [01:27<00:00,  2.81it/s]


Epoch 250/250: Train Loss: 0.3798, Train Accuracy: 82.4836, Train IOU: 0.6767, Test Loss: 0.3366, Test Accuracy: 85.2841, Test IOU: 0.7150


<All keys matched successfully>

In [13]:
torch.save(modelunetplusplus, 'unetplusplus_efficientnet_v3_effb4.pth')
#load model
#modelunet = torch.load('modelunet.pth')

In [14]:
import pandas as pd

trls = train_losses3
print(trls)
tracc = train_accuracies3
print(tracc)
tsls = test_losses3
print(tsls)
tsacc = test_accuracies3
print(tsacc)
triou = [t1.cpu().numpy() for t1 in train_ious3]
print(triou)
tsiou = [t2.cpu().numpy() for t2 in test_ious3]
print(tsiou)

# Create a dictionary to form the DataFrame
data = {
    'train_losses': trls,
    'train_accuracies': tracc,
    'test_losses': tsls,
    'test_accuracies': tsacc,
    'train_ious': triou,
    'test_ious': tsiou
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('performance_metrics_unetplusplus_efficientnet_v3_effb4.csv', index=False)

[0.5269540593149217, 0.5254906724865843, 0.527080292866482, 0.525242132263455, 0.523377465644503, 0.5210820550356454, 0.5192809751847896, 0.5202951014526491, 0.5205938080704309, 0.5181554483204354, 0.5181272539908324, 0.515219532256204, 0.5114972999667734, 0.5115081596422971, 0.511740880283883, 0.5109020511551601, 0.5097810112363924, 0.5104456606919203, 0.5072541514305564, 0.5070360141798733, 0.5094442378456999, 0.5064586667510552, 0.5020529654452471, 0.5072277436653773, 0.49940561484999774, 0.501272795040433, 0.5011446181593872, 0.5000932935049863, 0.49810845980314705, 0.4969224600287957, 0.495706641455976, 0.4981955615727882, 0.49592033670685154, 0.49275960815631276, 0.49464762041239235, 0.49306857343611676, 0.49023622771104175, 0.4883314017357865, 0.49452471660404673, 0.48859181551913905, 0.48848417219592305, 0.48743818809346456, 0.48331151939019923, 0.48930584778630637, 0.49072348550567785, 0.4836327534138672, 0.4853306644088854, 0.4835572180951514, 0.48119648854906966, 0.481952228

In [15]:
model_list = {'unet': model_unet, 'deeplabv3+': model_deeplabv3plus, 'pan': model_pan, 'manet':model_manet}

In [ ]:
# Define a function to train and evaluate a model
def train_and_evaluate(model, trainloader, testloader, optimizer, lossfn, epochs, device, patience=10):
    best_loss = float('inf')
    best_model_wts = None

    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []
    train_ious = []
    test_ious = []

    # Initialize EarlyStopping
    early_stopping = EarlyStopping(patience=patience)

    for epoch in range(epochs):
        train_loss, train_accuracy, train_iou = train(model, trainloader, optimizer, lossfn, device)
        test_loss, test_accuracy, test_iou = evaluate(model, testloader, lossfn, device)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        train_ious.append(train_iou.cpu().numpy())
        test_ious.append(test_iou.cpu().numpy())

        print(f'Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train IOU: {train_iou:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test IOU: {test_iou:.4f}')

        # Check early stopping
        early_stopping(test_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

        # Save the best model
        if test_loss < best_loss:
            best_loss = test_loss
            best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)

    return model, {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'test_losses': test_losses,
        'test_accuracies': test_accuracies,
        'train_ious': train_ious,
        'test_ious': test_ious
    }

In [ ]:
epochs = 250
patience = 10  # Set patience for early stopping
for model_name, model in model_list.items():
    print(f'Training {model_name}')
    model.to(DEVICE)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)  # Define optimizer for each model

    trained_model, metrics = train_and_evaluate(model, trainloader, testloader, optimizer, lossfn, epochs, DEVICE, patience)

    # Save the model weights
    torch.save(trained_model, f'{model_name}_efficientnet_best_model_v3_effb4.pth')

    # Save the metrics to a CSV file
    df = pd.DataFrame(metrics)
    df.to_csv(f'performance_metrics_{model_name}_v3_effb4.csv', index=False)

Training unet


  0%|          | 0/246 [00:00<?, ?it/s]c:\Users\sanji\anaconda3\Lib\site-packages\rasterio\__init__.py:317: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
100%|██████████| 246/246 [00:59<00:00,  4.14it/s]


Epoch 1/250: Train Loss: 0.6598, Train Accuracy: 61.6032, Train IOU: 0.3727, Test Loss: 6.1253, Test Accuracy: 54.7143, Test IOU: 0.1333


100%|██████████| 246/246 [00:59<00:00,  4.16it/s]


Epoch 2/250: Train Loss: 0.5934, Train Accuracy: 65.8285, Train IOU: 0.4089, Test Loss: 0.5710, Test Accuracy: 68.7144, Test IOU: 0.4524


...

Epoch 248/250: Train Loss: 0.3145, Train Accuracy: 86.0104, Train IOU: 0.7349, Test Loss: 0.2770, Test Accuracy: 87.5914, Test IOU: 0.7521


100%|██████████| 246/246 [01:06<00:00,  3.71it/s]


Epoch 249/250: Train Loss: 0.3182, Train Accuracy: 85.6854, Train IOU: 0.7287, Test Loss: 0.2783, Test Accuracy: 87.3969, Test IOU: 0.7471


100%|██████████| 246/246 [00:57<00:00,  4.24it/s]


Epoch 250/250: Train Loss: 0.3193, Train Accuracy: 85.7618, Train IOU: 0.7299, Test Loss: 0.2801, Test Accuracy: 87.9682, Test IOU: 0.7648


In [ ]:
import rasterio
from rasterio.transform import from_origin
from PIL import Image
import numpy as np

out_folder_img = './Data/Test_Results/Image'
out_folder_label = './Data/Test_Results/Label'

def save_image_rasterio(image, filepath):
    # image is expected to be of shape (channels, height, width)
    if image.ndim == 4 and image.shape[0] == 1:
        image = image[0]  # Remove the channel dimension if it's single channel

    height, width = image.shape[-2], image.shape[-1]
    transform = from_origin(0, 0, 1, 1)  # Modify as per your coordinate reference system

    with rasterio.open(
        filepath,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1 if image.ndim == 2 else image.shape[0],  # Set the count to number of channels
        dtype=image.dtype,
        crs='+proj=latlong',
        transform=transform,
    ) as dst:
        if image.ndim == 2:
            dst.write(image, 1)  # Assuming the image is single channel (grayscale)
        else:
            for band in range(image.shape[0]):
                dst.write(image[band], band + 1)  # Write each channel separately

def make(model, testloader, device):
    model.eval()
    predicted_images = []  # List to store predicted images
    input_images = []  # List to store input images

    with torch.no_grad():
        for batch_idx, (x_batch, y_batch) in enumerate(testloader):
            x_batch = x_batch.float().to(device)
            y_batch = y_batch.float().to(device)

            # Forward pass and loss calculation
            yhat = model(x_batch)
  
            # Predictions
            predicted_classes = (yhat > 0.5).float() # Round to get binary predictions
            
            # Saving predicted images
            for i in range(predicted_classes.size(0)):  # Iterate over the batch dimension
                predicted_image = predicted_classes[i].cpu().numpy()
                input_image = x_batch[i].cpu().numpy()
                save_image_rasterio(predicted_image, f"{out_folder_label}/prediction_{batch_idx}_{i}.tif")
                save_image_rasterio(input_image, f"{out_folder_img}/input_{batch_idx}_{i}.tif")
                #save_image_png(predicted_image, f"{out_folder_img}/prediction_{batch_idx}_{i}.png")
                #save_image_png(input_image, f"{out_folder_label}/input_{batch_idx}_{i}.png")
                predicted_images.append(predicted_image)  # Store the predicted image in the list
                input_images.append(input_image)  # Store the input image in the list

    return input_images, predicted_image


def save_image_png(image, filepath):
    # image is expected to be of shape (channels, height, width)
    if image.ndim == 3 and image.shape[0] == 1:
        image = image[0]  # Remove the channel dimension if it's single channel

    if image.ndim == 2:  # Single channel image
        image = (image * 255).astype(np.uint8)  # Convert to uint8
        Image.fromarray(image).save(filepath)
    else:  # Multi-channel image
        if image.shape[0] == 4:  # Drop the fourth channel if present
            image = image[:3, :, :]
        image = np.transpose(image, (1, 2, 0))  # Convert from (channels, height, width) to (height, width, channels)
        image = (image * 65535).astype(np.uint8)  # Convert to uint8
        Image.fromarray(image).save(filepath)

In [ ]:
import torch
import rasterio
modelunetplusplus = torch.load('modelunetplusplus_dynamicearthnet.pth')

inference_test = make(modelunetplusplus, testloader, DEVICE)